# Analysis

This analysis reproduces the analysis performed in:

> Monks T, Worthington D, Allen M, Pitt M, Stein K, James MA. A modelling tool for capacity planning in acute and community stroke services. BMC Health Serv Res. 2016 Sep 29;16(1):530. doi: [10.1186/s12913-016-1789-4](https://doi.org/10.1186/s12913-016-1789-4). PMID: 27688152; PMCID: PMC5043535.

It is organised into:

* Base case
    * Run the model
    * Figure 1
    * Theory: probability of delay
    * Figure 3
* Scenario analysis: altering arrivals
    * Scenario 1
    * Table 2
    * Scenario 4
    * Supplementary table 1
* Scenario analysis: pooling beds
    * Theory: pooling beds
    * Scenario 2

In [1]:
# pylint: disable=missing-module-docstring
%load_ext autoreload
%autoreload 1
%aimport simulation

# pylint: disable=wrong-import-position
import os
import time

from IPython.display import display
import numpy as np
import pandas as pd
import plotly.express as px

from simulation.parameters import Param, ASUArrivals, RehabArrivals
from simulation.runner import Runner

In [2]:
# Start timer
start_time = time.time()

In [3]:
# Path to the outputs folder
OUTPUT_DIR = "../outputs/"

## Base case

### Run the model

In [4]:
# Set up runner to run in parallel with nine cores
base_runner = Runner(param=Param(cores=9))

# Run the model for 150 replications
base_reps, base_overall, base_audit = base_runner.run_reps()

### Figure 1

**Figure 1.** Simulation probability density function for occupancy of an acute stroke unit.

In [5]:
def plot_occupancy_freq(df, unit, file, path=OUTPUT_DIR):
    """
    Plot the frequency at which each occupancy level was observed in the audit.

    Parameters
    ----------
    df: pd.DataFrame
        Dataframe output by `get_occupancy_freq()` containing the frequency
        each occupancy was observed at.
    unit: str
        Name of unit ("asu", "rehab")
    file: str
        Filename to save figure to (e.g. "figure.png").
    path: str
        Path to save file to (excluding filename).
    """
    # Create plot
    fig = px.bar(df, x="beds", y="pct", color_discrete_sequence=["black"])

    # Specify axis labels, theme and dimensions
    if unit == "asu":
        unit_lab = "acute"
    elif unit == "rehab":
        unit_lab = "rehabilitation"
    else:
        raise ValueError("unit must be either 'acute' or 'rehab'")

    fig.update_layout(
        xaxis_title=f"No. patients in {unit_lab} unit",
        yaxis_title="% observations",
        template="plotly_white",
        height=450,
        width=800
    )

    # Add box around figure, and set tick spacing to 1
    fig.update_xaxes(linecolor="black", mirror=True, dtick=1)
    fig.update_yaxes(linecolor="black", mirror=True, tickformat=",.0%")

    # Show figure
    fig.show()

    # Save figure
    fig.write_image(os.path.join(path, file))

**Generate plots...**

(the article just includes a plot for the acute stroke unit).

In [6]:
# Acute stroke unit
plot_occupancy_freq(base_overall["asu"], unit="asu",
                    file="figure1_asu.png")

# Rehabilitation unit
plot_occupancy_freq(base_overall["rehab"], unit="rehab",
                    file="figure1_rehab.png")

### Theory: probability of delay

We can use our frequency and cumulative frequency of occupied beds from the simulation to calculate blocking probability.

For example:

| Beds | `pct` | `c_pct` | Probability of delay |
| - | - | - | - |
| 0 | 0.2 | 0.2 | 1.0 |
| 1 | 0.3 | 0.5 | 0.6 |
| 2 | 0.4 | 0.9 | 0.44 |
| 3 | 0.1 | 1.0 | 0.1 |

We can interpret...

* `pct` as the probability of having exactly x beds occupied.
* `c_pct` as the probability of having x or fewer beds occupied.

We can then calculate `pct/c_pct`, which is the probability of delay when the system has exactly x beds occupied.

Interpretation for 1 bed:

* If we **randomly select a day when the occupancy is 1 or fewer beds**, there's a **60%** chance that the occupancy will be **exactly 1 beds (rather than 0 beds)**.

This can then be connected to the probability of delay by thinking about system capacity:

* If we assume that the unit has a total of 1 beds, then when 1 beds are occupied, the unit is at **full capacity**.
* Any new patients arriving when 1 beds are occupied would experience a delay.
* So 0.6 represents the probability that, given we're at or below capacity (0 or 1 beds), we're actually at full capacity (1 beds)

In other words, `pct/c_pct` is the probability that a new arrival will experience a delay when the system has exactly x beds occupied, given that the capacity of the system is x beds.

### Figure 3

**Figure 3**. Simulated trade-off between the probability that a patient is delayed and the no. of acute beds available.

In [7]:
def plot_delay_prob(df, unit, file, path=OUTPUT_DIR):
    """
    Plot the simulated trade-off between the probability of delay and the
    number of beds available.

    Parameters
    ----------
    df: pd.DataFrame
        Dataframe output by `get_occupancy_freq()` containing the frequency
        each occupancy was observed at.
    unit: str
        Name of unit ("asu", "rehab")
    file: str
        Filename to save figure to (e.g. "figure.png").
    path: str
        Path to save file to (excluding filename).
    """
    # Create the step plot
    fig = px.line(df, x="beds", y="prob_delay",
                  color_discrete_sequence=["black"])
    fig.update_traces(mode="lines", line_shape="hv")

    # Add axis labels, set theme and dimensions
    if unit == "asu":
        unit_lab = "acute"
    elif unit == "rehab":
        unit_lab = "rehabilitation"
    else:
        raise ValueError("unit must be either 'acute' or 'rehab'")

    fig.update_layout(
        xaxis_title=f"No. of {unit_lab} beds available",
        yaxis_title="Probability of delay",
        template="simple_white",
        height=450,
        width=800
    )

    # Set tick frequency and adjust axis
    fig.update_xaxes(dtick=1)
    fig.update_yaxes(dtick=0.1, range=[0, 1])

    # Show figure
    fig.show()

    # Save figure
    fig.write_image(os.path.join(path, file))

In [8]:
plot_delay_prob(base_overall["asu"], unit="asu", file="figure3_asu.png")
plot_delay_prob(base_overall["rehab"], unit="rehab", file="figure3_rehab.png")

## Scenario analysis: altering arrivals

### Scenario 1

**5% more admissions.** A 5% increase in admissions across all patient subgroups.

In [9]:
def alter_by_5_percent(params_dict):
    """
    Helper function to reduce all attributes of a class by 5%.

    Parameters
    ----------
    params_dict: dict
        Dictionary of parameters.
    """
    return {k: v * 0.95 for k, v in params_dict.items() if k != '_initialised'}


# Apply 5% increase to inter-arrival parameters
s1_param = Param(
    asu_arrivals=ASUArrivals(**alter_by_5_percent(vars(ASUArrivals()))),
    rehab_arrivals=RehabArrivals(**alter_by_5_percent(vars(RehabArrivals()))),
    cores=9
)

print(vars(s1_param.asu_arrivals))
print(vars(s1_param.rehab_arrivals))

{'stroke': 1.14, 'tia': 8.835, 'neuro': 3.42, 'other': 3.04, '_initialised': True}
{'stroke': 20.71, 'neuro': 30.115, 'other': 27.17, '_initialised': True}


In [10]:
# Run the model for 150 replications
s1_runner = Runner(param=s1_param)
s1_reps, s1_overall, s1_audit = s1_runner.run_reps()

### Table 2

**Table 2** Likelihood of delay. Current admissions versus 5% more admissions.

This table presents results from the base case and scenario 1 for acute beds 9-14 and rehab beds 10-16.

In [11]:
def make_delay_table(
    scenario, scenario_name, base, base_name, asu_beds, rehab_beds
):
    """
    Create table with the probability of delay and 1 in n patients delayed,
    for the base case and a provided scenario.

    Parameters
    ----------
    scenario: dict
        Dictionary containing two dataframes: "asu" and "rehab". These contain
        the overall results from a scenario run of the simulation.
    scenario_name: str
        Name for scenario to use in table labels.
    base: dict
        Dictionary containing two dataframes: "asu" and "rehab". These contain
        the overall results from the base case run of the simulation.
    base_name: str
        Name for base case to use in table labels.
    asu_beds: list
        List of acute stroke unit (ASU) bed numbers to get results for.
    rehab_beds: list
        List of rehabilitation unit bed numbers to get results for.
    """
    # Create list to store the ASU and rehab dataframes
    tab_full = []

    # Loop over ASU and rehab units...
    for unit_name, unit_beds in {"asu": asu_beds, "rehab": rehab_beds}.items():

        # Create list to store base case and scenario dataframes
        tab_segment = []

        # Loop over base case and scenario...
        for name, df in {base_name: base[unit_name],
                         scenario_name: scenario[unit_name]}.items():

            # Extract results for specified beds
            df = df[df["beds"].isin(unit_beds)][
                ["beds", "prob_delay", "1_in_n_delay"]]

            # Rename column to be specific to scenario
            df = df.rename(columns={
                "prob_delay": f"prob_delay_{name}",
                "1_in_n_delay": f"1_in_n_delay_{name}"})

            # Save dataframe to list
            tab_segment.append(df)

        # Combine into single dataframe
        full_df = pd.merge(tab_segment[0], tab_segment[1], on="beds")

        # Add column with unit name
        full_df.insert(0, "unit", unit_name)

        # Save dataframe to list
        tab_full.append(full_df)

    # Combine into a single table
    return pd.concat(tab_full).reset_index(drop=True)

In [12]:
full_tab2 = make_delay_table(
    scenario=s1_overall, scenario_name="5%", base=base_overall,
    base_name="current", asu_beds=list(range(9,15)),
    rehab_beds=list(range(10,17)))
display(full_tab2)

,unit,beds,prob_delay_current,1_in_n_delay_current,prob_delay_5%,1_in_n_delay_5%
0,asu,9,0.181649,6.0,0.203707,5.0
1,asu,10,0.128773,8.0,0.149243,7.0
2,asu,11,0.086540,12.0,0.104845,10.0
3,asu,12,0.056439,18.0,0.070083,14.0
4,asu,13,0.033222,30.0,0.043671,23.0
5,asu,14,0.019337,52.0,0.025647,39.0
6,rehab,10,0.205611,5.0,0.227895,4.0
7,rehab,11,0.153553,7.0,0.174892,6.0
8,rehab,12,0.110100,9.0,0.129832,8.0
9,rehab,13,0.076047,13.0,0.092773,11.0


These are some adjustments to how table is presented in article...

(hiding / dropping some results)

The 1 in n delay columns are **rounded to the nearest whole number**, but python doesn't allow NaN or Inf in an int column, so they provided as floats.

In [13]:
adj_full_tab_2 = full_tab2.copy()

# Round probability of delay to 2 d.p.
adj_full_tab_2["prob_delay_current"] = round(
    adj_full_tab_2["prob_delay_current"], 2)
adj_full_tab_2["prob_delay_5%"] = round(
    adj_full_tab_2["prob_delay_5%"], 2)

# Drop the result for ASU beds 9 and rehab beds 10 for the scenario
adj_full_tab_2.loc[(adj_full_tab_2["unit"] == "asu") &
                   (adj_full_tab_2["beds"] == 9),
                   ["prob_delay_5%", "1_in_n_delay_5%"]] = None
adj_full_tab_2.loc[(adj_full_tab_2["unit"] == "rehab") &
                   (adj_full_tab_2["beds"] == 10),
                   ["prob_delay_5%", "1_in_n_delay_5%"]] = None

# Drop the result for rehab 11 beds
adj_full_tab_2 = adj_full_tab_2[
    ~((adj_full_tab_2["unit"] == "rehab") & (adj_full_tab_2["beds"] == 11))]

# Display and save to csv
display(adj_full_tab_2)
adj_full_tab_2.to_csv(
    os.path.join(OUTPUT_DIR, "table2.csv"), index=False)

,unit,beds,prob_delay_current,1_in_n_delay_current,prob_delay_5%,1_in_n_delay_5%
0,asu,9,0.18,6.0,NaN,NaN
1,asu,10,0.13,8.0,0.15,7.0
2,asu,11,0.09,12.0,0.10,10.0
3,asu,12,0.06,18.0,0.07,14.0
4,asu,13,0.03,30.0,0.04,23.0
5,asu,14,0.02,52.0,0.03,39.0
6,rehab,10,0.21,5.0,NaN,NaN
8,rehab,12,0.11,9.0,0.13,8.0
9,rehab,13,0.08,13.0,0.09,11.0
10,rehab,14,0.05,20.0,0.06,16.0


### Scenario 4

**No complex-neurological cases.** Complex neurological patients are excluded from the pathway in order to assess their impact on bed requirements.

In [14]:
# Set IAT very high, essentially meaning that we have no neuro arrivals
s4_param = Param(
    asu_arrivals=ASUArrivals(neuro = 10_000_000_000),
    rehab_arrivals=RehabArrivals(neuro=10_000_000_000),
    cores=9
)

In [15]:
# Run the model for 150 replications
s4_runner = Runner(param=s4_param)
s4_reps, s4_overall, s4_audit = s4_runner.run_reps()

### Supplementary table 1

**Supplementary Table 1.** Likelihood of delay. Current admissions versus No Complex neurological patients.

In [16]:
# Make table
sup_tab1 = make_delay_table(scenario=s4_overall,
                            scenario_name="no_complex_neuro",
                            base=base_overall,
                            base_name="current",
                            asu_beds=list(range(10,16)),
                            rehab_beds=list(range(12,17)))

# Round the probability of delay to 2 d.p.
sup_tab1["prob_delay_current"] = round(sup_tab1["prob_delay_current"], 2)
sup_tab1["prob_delay_no_complex_neuro"] = round(
    sup_tab1["prob_delay_no_complex_neuro"], 2)

# Display and save to csv
display(sup_tab1)
sup_tab1.to_csv(os.path.join(OUTPUT_DIR, "suptable1.csv"), index=False)

,unit,beds,prob_delay_current,1_in_n_delay_current,prob_delay_no_complex_neuro,1_in_n_delay_no_complex_neuro
0,asu,10,0.13,8.0,0.08,12.0
1,asu,11,0.09,12.0,0.05,20.0
2,asu,12,0.06,18.0,0.03,36.0
3,asu,13,0.03,30.0,0.01,67.0
4,asu,14,0.02,52.0,0.01,131.0
5,asu,15,0.01,95.0,0.00,277.0
6,rehab,12,0.11,9.0,0.05,19.0
7,rehab,13,0.08,13.0,0.03,32.0
8,rehab,14,0.05,20.0,0.02,57.0
9,rehab,15,0.03,32.0,0.01,110.0


## Scenario analysis: pooling beds

### Scenario 2

Scenario 2: **Pooling of acute and rehab beds.** The acute and rehab wards are co-located at same site. Beds are pooled and can be used by either acute or rehabilitation patients. Pooling of the total bed stock of 22 is compared to the pooling of an increased bed stock of 26.

In [17]:
# Calculate the combined occupancy from each timepoint in the audit
combined_audit = [
    {"pooled_occupancy": b["asu_occupancy"] + b["rehab_occupancy"]}
    for b in base_audit
]

# Hijack the get_occupancy_freq() method from runner to calculate stats
pooled_results = base_runner.get_occupancy_freq(combined_audit, unit="pooled")
display(pooled_results)

,beds,freq,pct,c_pct,prob_delay,1_in_n_delay
0,3,3,0.000011,0.000011,1.000000,1.0
1,4,14,0.000051,0.000062,0.823529,1.0
2,5,51,0.000186,0.000248,0.750000,1.0
3,6,189,0.000690,0.000939,0.735409,1.0
4,7,466,0.001702,0.002641,0.644537,2.0
5,8,1178,0.004303,0.006944,0.619674,2.0
6,9,2324,0.008489,0.015434,0.550059,2.0
7,10,4106,0.014999,0.030433,0.492858,2.0
8,11,6748,0.024650,0.055083,0.447510,2.0
9,12,10234,0.037384,0.092468,0.404298,2.0


Get probability of delay and 1 in every n patients delayed from 22 and 26 pooled beds, to use in Table 3 below.

In [18]:
pdelay_pooling_22 = pooled_results[
    pooled_results["beds"] == 22]["prob_delay"].item()
npatients_pooling_22 = pooled_results[
    pooled_results["beds"] == 22]["1_in_n_delay"].item()
print(pdelay_pooling_22, npatients_pooling_22)

0.06326972210295412 16.0


In [19]:
pdelay_pooling_26 = pooled_results[
    pooled_results["beds"] == 26]["prob_delay"].item()
npatients_pooling_26 = pooled_results[
    pooled_results["beds"] == 26]["1_in_n_delay"].item()
print(pdelay_pooling_26, npatients_pooling_26)

0.01641204878140444 61.0


### Scenario 3

Scenario 3: **Partial pooling of acute and rehab beds.** The acute and rehab wards are co-located at same site. A subset of the 26 beds are pooled and can be used by either acute or rehab patients.

In [20]:
class PooledDelay:
    """
    Class to calculate probability of delays in scenarios with partial pooling
    of acute and rehab beds.

    Attributes
    ----------
    asu : pd.Series
        Frequency distribution of ASU bed occupancies.
    rehab : pd.Series
        Frequency distribution of rehab bed occupancies.
    asu_beds : int or float
        Number of dedicated ASU beds (excluding pooled beds).
    rehab_beds : int or float
        Number of dedicated rehab beds (excluding pooled beds).
    pooled_beds : int or float
        Number of beds that can be used by either unit.
    """
    def __init__(self, base_results):
        """
        Initialise the PooledDelay object with base simulation results.

        Parameters
        ----------
        base_results: dict
            Dictionary containing two dataframes: "asu" and "rehab". These
            contain the overall results from the base run of the simulation.
        """
        # Extract the ASU and rehab frequencies
        self.asu = base_results["asu"].set_index("beds")["freq"]
        self.rehab = base_results["rehab"].set_index("beds")["freq"]

        # To store the bed counts
        self.asu_beds = np.nan
        self.rehab_beds = np.nan
        self.pooled_beds = np.nan

    def prob_occupancy(self, occ_freq, threshold, comparison):
        """
        Calculate the probability of an occupancy based on the specified
        comparison with a threshold value, using observed frequencies of
        different occupancies.

        Parameters
        ----------
        occ_freq : pd.Series
            Frequencies of each number of beds, with the index representing the
            occupancy.
        threshold : int
            The threshold number of beds for comparison.
        comparison : str
            The type of comparison to perform. Options are: "ge" (greater than
            or equal to, >=), "eq" (equal to, =), or "lt" (less than, <).

        Returns
        -------
        float
            Probability that the occupancy meets the specified comparison
            condition.
        """
        # Calculate total frequency
        total_freq = occ_freq.sum()

        # Calculate the frequency based on the comparison type
        if comparison == "ge":
            # Greater than or equal to
            filtered_freq = occ_freq[occ_freq.index >= threshold].sum()
        elif comparison == "eq":
            # Equal to
            filtered_freq = occ_freq[occ_freq.index == threshold].sum()
        elif comparison == "lt":
            # Less than
            filtered_freq = occ_freq[occ_freq.index < threshold].sum()
        else:
            raise ValueError(f"Comparison '{comparison}' not valid.")

        # Calculate and return the probability
        return filtered_freq / total_freq

    def calculate_only_unit_overflow(self, unit):
        """
        Calculate the probability of only the specified unit having delays.

        Parameters
        ----------
        unit: str
            Name of unit to investigate ("asu", "rehab").

        Returns
        -------
        float
            Probability that only the specified unit has delays.
        """
        # Determine name of other unit, depending on which you are focussing on
        other_unit = "rehab" if unit == "asu" else "asu"

        # Get the counts of available beds for focus unit and other unit
        unit_beds = getattr(self, f"{unit}_beds")
        other_beds = getattr(self, f"{other_unit}_beds")

        # Only that unit will have delays if they >= dedicated + pooled,
        # whilst other unit < dedicated
        p_unit = self.prob_occupancy(
            occ_freq=getattr(self, unit),
            threshold=unit_beds+self.pooled_beds,
            comparison="ge"
        )
        p_other = self.prob_occupancy(
            occ_freq=getattr(self, other_unit),
            threshold=other_beds,
            comparison="lt"
        )
        return p_unit*p_other

    def get_delay_combinations(self, unit):
        """
        Generates a list of dictionaries representing combinations of patients
        that would cause delays.

        This function calculates all combinations where the number of patients
        for a specific unit exceeds its dedicated beds but stays within the
        maximum capacity (dedicated + pooled beds).

        Parameters
        ----------
        unit: str
            Name of unit to investigate ("asu", "rehab").

        Returns
        -------
        combinations: list
            List of dictionaries containing the asu and rehab bed counts.
        """
        # Determine name of other unit, depending on which you are focussing on
        other_unit = "rehab" if unit == "asu" else "asu"

        # Get the counts of available beds for focus unit and other unit
        unit_beds = getattr(self, f"{unit}_beds")
        other_beds = getattr(self, f"{other_unit}_beds")

        # Find max beds for that unit
        max_beds = unit_beds + self.pooled_beds

        # Get combinations of beds which would cause a delay
        combinations = [{unit: i, other_unit: max_beds - i + other_beds}
                       for i in range(unit_beds, max_beds + 1)][::-1]

        return combinations

    def calculate_combination_probability(self, combination):
        """
        Calculate the probability of a set of combinations occurring.

        Parameters
        ----------
        combination: dict
            Dictionary with keys "asu" and "rehab" specifying thresholds.
            This must be that output from get_delay_combinations(), which
            outputs them in descending order for the unit capacity, which is
            important, as the first we check >=, and others ==.

        Returns
        -------
        float
            Probability of this combination occurring.
        """
        # List to store results
        probabilities = []

        # Use a counter to loop through the combinations...
        for i, combo in enumerate(combination):

            # Extract the primary unit and other unit's name and thresholds
            unit_name, other_name = combo.keys()
            unit_threshold, other_threshold = combo.values()

            # If it's the first combination, then we check unit >= threshold
            if i == 0:
                p_unit = self.prob_occupancy(
                    occ_freq=getattr(self, unit_name),
                    threshold=unit_threshold,
                    comparison="ge")
            # For all other items, we check unit == threshold
            else:
                p_unit = self.prob_occupancy(
                    occ_freq=getattr(self, unit_name),
                    threshold=unit_threshold,
                    comparison="eq")

            # The other unit will always be checking >= threshold
            p_other = self.prob_occupancy(
                occ_freq=getattr(self, other_name),
                threshold=other_threshold,
                comparison="ge")

            # Multiply probabilities and add to list
            probabilities.append(p_unit*p_other)

        # Return sum of list
        return sum(probabilities)

    def calculate_delay(self, asu_beds, rehab_beds, pooled_beds):
        """
        Analyse a bed pooling scenario.

        Parameters
        ----------
        asu_beds: int
            Number of dedicated ASU beds (excluding pooled beds).
        rehab_beds: int
            Number of dedicated rehabilitation beds (excluding pooled beds).
        pooled_beds: int
            Number of beds that can be used by either unit.

        Returns
        -------
        pool_results: dict
            Dictionary containing number of dedicated and pooled beds,
            probability of delay for each unit, and 1 in n patients delayed
            for each unit.
        """
        # Get counts of dedicated and pooled beds
        self.asu_beds = asu_beds
        self.rehab_beds = rehab_beds
        self.pooled_beds = pooled_beds

        # Get probability of only the ASU, or only rehab, having delays
        p_asu_only = self.calculate_only_unit_overflow(unit="asu")
        #print(f"p_asu_only {p_asu_only}")
        p_rehab_only = self.calculate_only_unit_overflow(unit="rehab")
        #print(f"p_rehab_only {p_rehab_only}")

        # Find combinations of patients that would cause delays
        asu_comb = self.get_delay_combinations(unit="asu")
        #print(f"asu combinations: {asu_comb}")
        rehab_comb = self.get_delay_combinations(unit="rehab")
        #print(f"rehab combinations: {rehab_comb}")

        # Convert those to probabilities
        p_asu_comb = self.calculate_combination_probability(asu_comb)
        p_rehab_comb = self.calculate_combination_probability(rehab_comb)

        # Create dictionary with the bed numbers, and the combined only +
        # combination probabilities for each unit
        pool_results = {
            "dedicated_acute": self.asu_beds,
            "dedicated_rehab": self.rehab_beds,
            "pooled": self.pooled_beds,
            "pdelay_acute": p_asu_only + p_asu_comb,
            "pdelay_rehab": p_rehab_only + p_rehab_comb}

        # Calculate 1 in every n patients delays
        pool_results["1_in_n_delay_acute"] = (
            round(1 / pool_results["pdelay_acute"]))
        pool_results["1_in_n_delay_rehab"] = (
            round(1 / pool_results["pdelay_rehab"]))

        # Round the probability of delay (after calculation, so doesn't impact)
        pool_results["pdelay_acute"] = round(pool_results["pdelay_acute"], 3)
        pool_results["pdelay_rehab"] = round(pool_results["pdelay_rehab"], 3)

        return pool_results

Create list with the 22 + 26 pooled bed results.

In [21]:
def create_pool_result(pooled_value, pdelay_value, npatients_value):
    """
    Create a dictionary representing a pool result entry.

    Parameters
    ----------
    pooled_value : int
        The value for the 'pooled' key.
    pdelay_value : float or int
        The probability of delay (for acute and rehab).
    npatients_value : int
        1 in every n patients delayed (for acute and rehab).

    Returns
    -------
    dict
        A dictionary with keys 'dedicated_acute', 'dedicated_rehab', 'pooled', 
        'pdelay_acute', and 'pdelay_rehab'.
    """
    return {
        "dedicated_acute": 0,
        "dedicated_rehab": 0,
        "pooled": pooled_value,
        "pdelay_acute": pdelay_value,
        "pdelay_rehab": pdelay_value,
        "1_in_n_delay_acute": npatients_value,
        "1_in_n_delay_rehab": npatients_value
    }


pool_result_list = [
    create_pool_result(22, pdelay_pooling_22, npatients_pooling_22),
    create_pool_result(26, pdelay_pooling_26, npatients_pooling_26)
]

Add result for 14 acute 12 rehab 0 pooled, as in table 2 above.

In [22]:
# Extract result for 14 acute and 12 rehab beds
acute14 = full_tab2[(full_tab2["unit"] == "asu") & (full_tab2["beds"] == 14)]
rehab12 = full_tab2[(full_tab2["unit"] == "rehab") & (full_tab2["beds"] == 12)]

pool_result_list.append({
    "dedicated_acute": 14,
    "dedicated_rehab": 12,
    "pooled": 0,
    "pdelay_acute": acute14["prob_delay_current"].item(),
    "pdelay_rehab": rehab12["prob_delay_current"].item(),
    "1_in_n_delay_acute": acute14["1_in_n_delay_current"].item(),
    "1_in_n_delay_rehab":rehab12["1_in_n_delay_current"].item()
})

Calculate results from other pooling scenarios and add to the list, then display as a dataframe.

In [23]:
# Create instance of class
pooled_delay = PooledDelay(base_results=base_overall)

# Loop through other pooling scenarios
for beds in [(11, 11, 4), (11, 10, 5), (10, 10, 6), (10, 9, 7),
             (9, 9, 8), (9, 8, 9)]:
    pool_result_list.append(pooled_delay.calculate_delay(
        asu_beds=beds[0], rehab_beds=beds[1], pooled_beds=beds[2]))

### Table 3

In [24]:
# Convert to a dataframe
tab3 = pd.DataFrame(pool_result_list)

for delay_type in ["acute", "rehab"]:
    # Round probabilities to 3 decimal places
    tab3[f"pdelay_{delay_type}"] = round(tab3[f"pdelay_{delay_type}"], 3)
    # Convert 1-in-n counts to integers
    tab3[f"1_in_n_delay_{delay_type}"] = (
        tab3[f"1_in_n_delay_{delay_type}"].astype(int))

# Display and save to csv
display(tab3)
tab3.to_csv(
    os.path.join(OUTPUT_DIR, "table3.csv"), index=False)

,dedicated_acute,dedicated_rehab,pooled,pdelay_acute,pdelay_rehab,1_in_n_delay_acute,1_in_n_delay_rehab
0,0,0,22,0.063,0.063,16,16
1,0,0,26,0.016,0.016,61,61
2,14,12,0,0.019,0.110,52,9
3,11,11,4,0.044,0.089,23,11
4,11,10,5,0.037,0.090,27,11
5,10,10,6,0.041,0.066,24,15
6,10,9,7,0.039,0.066,26,15
7,9,9,8,0.042,0.053,24,19
8,9,8,9,0.041,0.053,25,19


## Run time

In [25]:
# Get run time in seconds
end_time = time.time()
runtime = round(end_time - start_time)

# Display converted to minutes and seconds
print(f'Notebook run time: {runtime // 60}m {runtime % 60}s')

Notebook run time: 0m 14s
